In [1]:
%matplotlib qt

import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d

In [2]:
LABELS = ["Alt_Si3N4", "Anc_Si3N4", "lambda", "n_eff", "A_eff"]

# --- función para normalizar dataset ---
def load_and_normalize(fname):
    with h5py.File(fname, "r") as f:
        A = f["/A"][...]
    axis5 = next((ax for ax, s in enumerate(A.shape) if s == 5), None)
    if axis5 is None:
        raise ValueError(f"No encontré eje de tamaño 5 en {fname}")
    if axis5 != A.ndim - 1:
        A = np.moveaxis(A, axis5, -1)
    return A

In [3]:
# --- cargar los 4 datasets ---
archivos = [
    "Datos_reto/Datos1.h5",
    "Datos_reto/Datos2.h5",
    "Datos_reto/Datos3.h5",
    "Datos_reto/Datos4.h5",
    "Datos_reto/Datos5.h5"
]
datasets = [load_and_normalize(fname) for fname in archivos]

# --- validación de bordes ---
def check_borders(d1, d2):
    alt1_end = np.round(d1[-1,0,0,0],6)
    alt2_start = np.round(d2[0,0,0,0],6)
    anc1_end = np.round(d1[0,-1,0,1],6)
    anc2_start = np.round(d2[0,0,0,1],6)
    print(f"Check Alt: {alt1_end} vs {alt2_start}, Check Anc: {anc1_end} vs {anc2_start}")
    if not np.isclose(alt1_end, alt2_start) and not np.isclose(anc1_end, anc2_start):
        print("⚠️ Advertencia: los bordes no coinciden perfectamente")

for i in range(len(datasets)-1):
    check_borders(datasets[i], datasets[i+1])

# --- unir todos en un bloque continuo ---
DatosU = np.block([
    [datasets[0]],
    [datasets[1]],
    [datasets[2]],
    [datasets[3]],
    [datasets[4]]
])

Check Alt: 0.4 vs 0.55, Check Anc: 0.875 vs 0.875
Check Alt: 0.55 vs 0.7, Check Anc: 1.25 vs 1.25
Check Alt: 0.7 vs 0.85, Check Anc: 1.625 vs 1.625
Check Alt: 0.85 vs 0.925, Check Anc: 2.0 vs 1.625
⚠️ Advertencia: los bordes no coinciden perfectamente


In [4]:
with h5py.File("Datos_reto/DatosU.h5", "w") as f_out:
    f_out.create_dataset("A", data=DatosU)

print("✅ Dataset unificado guardado en DatosU.h5, shape:", DatosU.shape)

✅ Dataset unificado guardado en DatosU.h5, shape: (50, 10, 40, 5)


In [5]:
def to_dataframe(arr):
    nA, nB, nC, _ = arr.shape
    rows = []
    for ii in range(nA):
        for jj in range(nB):
            for kk in range(nC):
                rec = {"i": ii+1, "j": jj+1, "k": kk+1}
                for p, name in enumerate(LABELS):
                    rec[name] = arr[ii, jj, kk, p]
                rows.append(rec)
    return pd.DataFrame(rows, columns=["i","j","k"]+LABELS)

df = to_dataframe(DatosU)
df["lambda_nm"] = df["lambda"] * 1000  # mantener referencia en nm
print(df)

        i   j   k  Alt_Si3N4  Anc_Si3N4  lambda     n_eff         A_eff  \
0       1   1   1   0.400000        0.5    0.72  1.801107  1.766958e-13   
1       1   1   2   0.416667        0.5    0.72  1.807751  1.808637e-13   
2       1   1   3   0.433333        0.5    0.72  1.813994  1.852233e-13   
3       1   1   4   0.450000        0.5    0.72  1.819495  1.894631e-13   
4       1   1   5   0.466667        0.5    0.72  1.826820  1.984774e-13   
...    ..  ..  ..        ...        ...     ...       ...           ...   
19995  50  10  36   0.925000        2.0    0.89  1.966443  1.032798e-12   
19996  50  10  37   0.943750        2.0    0.89  1.967794  1.049798e-12   
19997  50  10  38   0.962500        2.0    0.89  1.969078  1.066814e-12   
19998  50  10  39   0.981250        2.0    0.89  1.970300  1.083843e-12   
19999  50  10  40   1.000000        2.0    0.89  1.971440  1.100531e-12   

       lambda_nm  
0          720.0  
1          720.0  
2          720.0  
3          720.0  
4   

In [8]:
fig1 = plt.figure(figsize=(10,8))
ax1 = fig1.add_subplot(111, projection='3d')
p1 = ax1.scatter(df["Alt_Si3N4"], df["Anc_Si3N4"], df["lambda"],
                 c=df["n_eff"], cmap="viridis", s=10)
ax1.set_xlabel("Altura (µm)")
ax1.set_ylabel("Ancho (µm)")
ax1.set_zlabel("Lambda (µm)")
fig1.colorbar(p1, ax=ax1, label="n_eff")
ax1.set_title("n_eff")

fig2 = plt.figure(figsize=(10,8))
ax2 = fig2.add_subplot(111, projection='3d')
p2 = ax2.scatter(df["Alt_Si3N4"], df["Anc_Si3N4"], df["lambda"],
                 c=df["A_eff"], cmap="plasma", s=10)
ax2.set_xlabel("Altura (µm)")
ax2.set_ylabel("Ancho (µm)")
ax2.set_zlabel("Lambda (µm)")
fig2.colorbar(p2, ax=ax2, label="A_eff")
ax2.set_title("A_eff")

plt.show()

In [7]:
# rangos en µm
rango_p = (0.720, 0.890)   # bomba
rango_s = (1.530, 1.570)   # señal
rango_i = (0.500, 0.680)   # idler

# todas las lambdas de bombeo dentro del rango
lambdas_p = df["lambda"].unique()
lambdas_p = sorted([lp for lp in lambdas_p if rango_p[0] <= lp <= rango_p[1]])

print(f"Voy a probar con {len(lambdas_p)} Lp entre {lambdas_p[0]} y {lambdas_p[-1]} µm")

Voy a probar con 50 Lp entre 0.72 y 0.89 µm


In [8]:
resultados = []

df_filtro = df[(df["i"] == 1) & (df["j"] == 1)].sort_values("lambda")
f_interp = interp1d(df_filtro["lambda"], df_filtro["n_eff"],
                    kind="linear", fill_value="extrapolate")

# barrido
for lp in lambdas_p:
    for ls in np.linspace(rango_s[0], rango_s[1], 500):
        if (1/lp - 1/ls) <= 0:
            continue
        li = 1 / (1/lp - 1/ls)
        if rango_i[0] <= li <= rango_i[1]:
            neff_s = float(f_interp(ls))
            neff_i = float(f_interp(li))
            resultados.append({
                "lambda_p (µm)": lp,
                "lambda_s (µm)": ls,
                "lambda_i (µm)": li,
                "n_eff_s": neff_s,
                "n_eff_i": neff_i
            })

tabla = pd.DataFrame(resultados)
print("=== Tabla phase matching ===")
print(tabla)

=== Tabla phase matching ===
Empty DataFrame
Columns: []
Index: []
